In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 15
TITLE = "30-31 March 2022"
SUBTITLE = "8 AM 31 March 2022"
SETPOINT_LOCS = {}
sts = utc(2022, 3, 31, 0)
ets = utc(2022, 3, 31, 20, 0)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        geo = gpd.points_from_xy(x=[SETPOINT_LOCS[sp][0]], y=[SETPOINT_LOCS[sp][1]], crs=df.crs)
        df.loc[10000 + sp] = {"geo": geo[0], "val": val, "used_for_analysis": True,
                             "plotme": True}
    df["geo"] = gpd.GeoSeries(df["geo"], crs=df.crs)

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [9]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = ['FSCI4', 'CHRI4', 'GLDI2', 'LSXI4']
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=2, upper=4)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='220401.png')
    mp.close()

main()

     state  wfo  val       lon      lat                              geo  \
0       NM  ABQ  7.0 -105.6300  36.0700  POINT (-507092.447 -975023.086)   
1       SD  ABR  4.5  -97.7100  45.9000    POINT (177180.331 102588.232)   
2       MI  MQT  4.1  -88.6700  47.0400    POINT (855125.211 287340.693)   
3       NM  ABQ  4.0 -105.1700  36.4600  POINT (-463285.422 -934487.803)   
4       KS  ICT  4.0  -97.5400  38.2000   POINT (215303.363 -752509.788)   
...    ...  ...  ...       ...      ...                              ...   
2282    NY  ALY  0.0  -73.4000  43.5500   POINT (2096335.187 186948.033)   
2283    NY  BGM  0.0  -76.5447  42.9327    POINT (1877140.584 43186.829)   
2284    NY  BGM  0.0  -76.6591  42.7338    POINT (1874453.443 19076.936)   
2285    NY  BGM  0.0  -76.3319  43.1569    POINT (1886519.141 72313.634)   
2286    NY  BGM  0.0  -75.3697  43.4361   POINT (1951147.883 125238.997)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
0                  True  